**Background:**

The purpose of this notebook is to transform a .wav file into a spectrogram and then feed it into a well known model such as InceptionV3 for feature generation

## Libraries and File Locations

In [1]:
import sys
sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import soundfile
from scipy import signal
import librosa
import matplotlib.pyplot as plt
import specdisplay

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:


## Define file generation parameters

**Number of Mel-frequencies to keep in the spectrograms**

In [3]:
n_mels = 128

## Defining spectrogram transformation function

## NOTE!!

We're averaging across channels, come back and try a different approach if that's not working

In [4]:
def extract_spectrogram(samples,sample_rate,n_mels=128,n_fft=2048):
    feature_set = []
    if samples.shape[1:]:
        channels = samples.shape[1:][0]
    else:
        channels = 1
        samples = np.expand_dims(samples, axis=1)
    for i in range(channels):
        sample_channel_x = samples[:,i]
        mel_spectrogram = librosa.feature.melspectrogram(y=sample_channel_x, 
                                                     sr=sample_rate,
                                                     n_fft=n_fft, 
                                                     n_mels = n_mels
                                                        )
        decibel_spec = librosa.logamplitude(mel_spectrogram,ref_power=np.max)
        feature_set.append(decibel_spec)
    feature_set = np.array(feature_set)
    feature_set = np.mean(feature_set,axis=0)
    return feature_set

# Creating the PNG spectrograms

**How are we organizing the output files?**

According to the researchers how created the dataset, larger files were used to obtain smaller audio sample for specific classes. With that in mind, having different chuncks of the same file in the training and validation datasets would cause the model to overfit and inflate validation accuracies. As a result, the researchers split the dataset into 10 folds, keeping data originated from the same files within a fold. The recomended way of working with the dataset is to use 9 out of the 10 folds for training and one for validation. This would be repeated 10 times with each fold being held as validation - (essentially k-fold).

The accuracy score should be reported by averaging the results of the 10 different training/fold exercises.


See more on this article:
https://urbansounddataset.weebly.com/urbansound8k.html

**Folder structure**

/fold/class/filename.png

This will help when using Kera's ImageDataGenerator.flow_from_directory api to train and test the data

In [5]:
urban_sounds_folder = "/media/romulo/6237-3231/fsdkaggle/"

In [6]:
metadata = pd.read_csv(urban_sounds_folder+'FSDKaggle2018.meta/test_post_competition_scoring_clips.csv')

In [7]:
data_folder = "FSDKaggle2018.audio_test"

In [8]:
for index, row in metadata[::].iterrows():
    print("Transforming file {}".format(index))
    
    if not os.path.exists(urban_sounds_folder+"/" + data_folder + "/"+'spectrograms'):
        os.makedirs(urban_sounds_folder+"/" + data_folder + "/"+'spectrograms')
    
    if not os.path.exists(urban_sounds_folder+"/" + data_folder + "/"+'spectrograms/' + row['label']):
        os.makedirs(urban_sounds_folder+"/" + data_folder + "/"+'spectrograms/' + row['label'])

    # Convert to log scale (dB). We'll use the peak power as reference.
    y, sr = soundfile.read(urban_sounds_folder+"/" + data_folder + "/"+ str(row['fname']))
    log_s = extract_spectrogram(y,sr,n_mels=n_mels,n_fft=2048)
    #new plot
    fig = plt.figure(figsize=(12,4))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    #getting spectrogram
    specdisplay.specshow(log_s, sr=sr, x_axis='time', y_axis='mel')

    #Saving PNG
    plt.savefig(urban_sounds_folder+"/" + data_folder + "/"+'spectrograms/' + row['label'] + 
                '/' + row['fname'] + '.png')
    plt.close()

../specdisplay.py:676: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(data.dtype, np.complex):


Transforming file 0
Transforming file 1
Transforming file 2
Transforming file 3
Transforming file 4
Transforming file 5
Transforming file 6
Transforming file 7
Transforming file 8
Transforming file 9
Transforming file 10
Transforming file 11
Transforming file 12
Transforming file 13
Transforming file 14
Transforming file 15
Transforming file 16
Transforming file 17
Transforming file 18
Transforming file 19
Transforming file 20
Transforming file 21
Transforming file 22
Transforming file 23
Transforming file 24
Transforming file 25
Transforming file 26
Transforming file 27
Transforming file 28
Transforming file 29
Transforming file 30
Transforming file 31
Transforming file 32
Transforming file 33
Transforming file 34
Transforming file 35
Transforming file 36
Transforming file 37
Transforming file 38
Transforming file 39
Transforming file 40
Transforming file 41
Transforming file 42
Transforming file 43
Transforming file 44
Transforming file 45
Transforming file 46
Transforming file 47
Tr

Transforming file 378
Transforming file 379
Transforming file 380
Transforming file 381
Transforming file 382
Transforming file 383
Transforming file 384
Transforming file 385
Transforming file 386
Transforming file 387
Transforming file 388
Transforming file 389
Transforming file 390
Transforming file 391
Transforming file 392
Transforming file 393
Transforming file 394
Transforming file 395
Transforming file 396
Transforming file 397
Transforming file 398
Transforming file 399
Transforming file 400
Transforming file 401
Transforming file 402
Transforming file 403
Transforming file 404
Transforming file 405
Transforming file 406
Transforming file 407
Transforming file 408
Transforming file 409
Transforming file 410
Transforming file 411
Transforming file 412
Transforming file 413
Transforming file 414
Transforming file 415
Transforming file 416
Transforming file 417
Transforming file 418
Transforming file 419
Transforming file 420
Transforming file 421
Transforming file 422
Transformi

Transforming file 752
Transforming file 753
Transforming file 754
Transforming file 755
Transforming file 756
Transforming file 757
Transforming file 758
Transforming file 759
Transforming file 760
Transforming file 761
Transforming file 762
Transforming file 763
Transforming file 764
Transforming file 765
Transforming file 766
Transforming file 767
Transforming file 768
Transforming file 769
Transforming file 770
Transforming file 771
Transforming file 772
Transforming file 773
Transforming file 774
Transforming file 775
Transforming file 776
Transforming file 777
Transforming file 778
Transforming file 779
Transforming file 780
Transforming file 781
Transforming file 782
Transforming file 783
Transforming file 784
Transforming file 785
Transforming file 786
Transforming file 787
Transforming file 788
Transforming file 789
Transforming file 790
Transforming file 791
Transforming file 792
Transforming file 793
Transforming file 794
Transforming file 795
Transforming file 796
Transformi

Transforming file 1120
Transforming file 1121
Transforming file 1122
Transforming file 1123
Transforming file 1124
Transforming file 1125
Transforming file 1126
Transforming file 1127
Transforming file 1128
Transforming file 1129
Transforming file 1130
Transforming file 1131
Transforming file 1132
Transforming file 1133
Transforming file 1134
Transforming file 1135
Transforming file 1136
Transforming file 1137
Transforming file 1138
Transforming file 1139
Transforming file 1140
Transforming file 1141
Transforming file 1142
Transforming file 1143
Transforming file 1144
Transforming file 1145
Transforming file 1146
Transforming file 1147
Transforming file 1148
Transforming file 1149
Transforming file 1150
Transforming file 1151
Transforming file 1152
Transforming file 1153
Transforming file 1154
Transforming file 1155
Transforming file 1156
Transforming file 1157
Transforming file 1158
Transforming file 1159
Transforming file 1160
Transforming file 1161
Transforming file 1162
Transformin

Transforming file 1477
Transforming file 1478
Transforming file 1479
Transforming file 1480
Transforming file 1481
Transforming file 1482
Transforming file 1483
Transforming file 1484
Transforming file 1485
Transforming file 1486
Transforming file 1487
Transforming file 1488
Transforming file 1489
Transforming file 1490
Transforming file 1491
Transforming file 1492
Transforming file 1493
Transforming file 1494
Transforming file 1495
Transforming file 1496
Transforming file 1497
Transforming file 1498
Transforming file 1499
Transforming file 1500
Transforming file 1501
Transforming file 1502
Transforming file 1503
Transforming file 1504
Transforming file 1505
Transforming file 1506
Transforming file 1507
Transforming file 1508
Transforming file 1509
Transforming file 1510
Transforming file 1511
Transforming file 1512
Transforming file 1513
Transforming file 1514
Transforming file 1515
Transforming file 1516
Transforming file 1517
Transforming file 1518
Transforming file 1519
Transformin